***Participantes (RM - NOME):***<br>
xxxx - xxxxx<br>
xxxx - xxxxx<br>
xxxx - xxxxx<br>
xxxx - xxxxx<br>

###**Criar um classificador de sentimento aplicando técnicas de PLN**
---

Utilizando o dataset de revisões de filmes em português [1], criar um classificador de sentimentos que consiga um score na métrica F1 Score superior a 70%.

Devem utilizar uma amostra de 20% e randon_state igual a 42 para testar as implementações e mensurar a métrica F1 Score (usar o parâmetro average = 'weighted') o restante dos dados devem ser utilizados para o treinamento (80%).

Fique a vontade para testar os métodos de pré-processamento, abordagens, algoritmos e bibliotecas, mas explique e justifique suas decisões.
O trabalho poderá ser feito em grupo de até 4 pessoas (mesmo grupo do Startup One).

Separe a implementação do seu modelo campeão junto com a parte de validação/teste de forma que o professor consiga executar todo o pipeline do modelo campeão.

Composição da nota:
- 50% - Demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, etc.)
- 50% - Baseado na performance obtida com o dataset de teste (conforme recomendação da amostra) no seu modelo campeão e na validação que o professor processar (Métrica F1 Score).

[1] - https://dados-ml-pln.s3-sa-east-1.amazonaws.com/reviews-pt-br.csv

Bom desenvolvimento!

In [4]:
import nltk
import spacy #!pip install spacy

import pandas as pd
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import MultinomialNB


In [6]:
# CARREGANDO O DATA FRAME
import pandas as pd
df = pd.read_csv('https://dados-ml-pln.s3-sa-east-1.amazonaws.com/reviews-pt-br.csv', encoding='utf-8')

# Façam o download do arquivo e utilizem localmente durante os testes

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44514 entries, 0 to 44513
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   codigo      44514 non-null  int64 
 1   texto       44514 non-null  object
 2   sentimento  44514 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [17]:
# Carrega StopWords
nlp = spacy.load('pt_core_news_sm') #python3 -m spacy download pt_core_news_sm
nltk.download('stopwords')

# stopwords NLTK
stops_nltk = nltk.corpus.stopwords.words('portuguese')

# stopwords SpaCy
stops_spacy = nlp.Defaults.stop_words

# stopwords do SpaCy e NLTK combinadas
stops = list(set(stops_spacy).union(set(stops_nltk)))
len(stops)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/njaneto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


499

In [18]:
# função de lematização completa do documento
def fn_lematiza_texto(texto):
    sent = []
    doc = nlp(texto)
    for word in doc:
        sent.append(word.lemma_)
        
    return ' '.join(sent)

# função de lematização dos verbos do documento
def fn_lematiza_verb_texto(texto):
    sent = []
    doc = nlp(texto)
    for word in doc:
        if word.pos_ =='VERB':
            sent.append(word.lemma_)
        else:
            sent.append(word.text)
    return " ".join(sent)


# função para limpar documento
def fn_limpa_texto(texto):
    texto = re.sub(r'@\w+','',texto)
    texto = re.sub(r'#','',texto)
    texto = re.sub(r'RT[\s]+','',texto)  
    #texto = re.sub(r'rt[\s]+','',texto)  
    texto = re.sub(r'https:/\/\S+','',texto)
    texto = re.sub(r'[,.:;]','',texto)
    texto = re.sub(r'\n',' ',texto)
    texto = re.sub(u'[^a-zA-ZâêîôûÂÊÎÔÛáéíóúÁÉÍÓÚàèìòùÀÈÌÒÙãõÃÕçÇ ]', '', texto)
    texto = texto.strip().lower()
    
    return texto

In [20]:
# aplica normalização no dataframe
df['texto_trat'] = df.texto.apply(fn_lematiza_texto)

# aplica a lematização no dataframe
df['texto_lemma'] = df.texto_trat.apply(fn_lematiza_texto)

# aplica a lematização dos verbos no dataframe
df['texto_lemma_verb'] = df.texto_trat.apply(fn_lematiza_verb_texto)

df.dropna(inplace=True)



In [ ]:
# analisa alguns dados
print(df['texto'][1])
print(df['texto_trat'][1])
print(df['texto_lemma'][1])
print(df['texto_lemma_verb'][1])

In [ ]:
# pré-processamento vetorização e BOW
# Frequência de termos - term frequency–inverse document frequency (TF-IDF)
vetor = TfidfVectorizer(ngram_range=(1,2), use_idf=True) #0.7729036810537457 - TF-IDF, texto lematixado, uni e bi brama, Naive

In [ ]:
# texto tratado
vetor.fit(df.texto_trat)
text_vect = vetor.transform(df.texto_trat)

In [ ]:
# texto tratado e lematizado
vetor.fit(df.texto_lemma)
text_vect = vetor.transform(df.texto_lemma)


In [ ]:
# texto tratado e verbos lematizado
vetor.fit(df.texto_lemma_verb)
text_vect = vetor.transform(df.texto_lemma_verb)

In [ ]:
# separa as amostras de Treino (80%) e Texte (20%)
X_train,X_test,y_train,y_test = train_test_split( text_vect,  df['sentimento'], test_size = 0.2, random_state = 42 )

# define e treina o modelo de classificação
#modelo = LogisticRegression(random_state=42)
#modelo = DecisionTreeClassifier(random_state=42)
modelo = MultinomialNB()

#modelo = LogisticRegression(penalty = 'l2', C = 100, random_state=42)

modelo.fit(X_train, y_train)

# valida a parformance
y_prediction = modelo.predict(X_test)
accuracy = accuracy_score(y_prediction, y_test)

print(text_vect.shape)
print(accuracy)

####**Validação do professor**

Consolidar apenas os scripts do seu **modelo campeão**, desde o carregamento do dataframe, separação das amostras, tratamentos utilizados (funções, limpezas, etc.), criação dos objetos de vetorização dos textos e modelo treinado e outras implementações utilizadas no processo de desenvolvimento do modelo.